In [1]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd

In [67]:
train = pd.read_csv("fashion-mnist_train.csv")
trainX = train.loc[:,train.columns != 'label'].to_numpy()

In [90]:
test = pd.read_csv("fashion-mnist_test.csv")
testX = test.loc[:,test.columns != 'label'].to_numpy()

In [9]:
# To be applied on RGB Images before using HOG
def convert_To_Greyscale(im1):
    image = ImageOps.grayscale(im1.resize((28,28))) 
 
    image_array = np.asarray(image,dtype=float)
    
    return image_array

In [11]:
image = convert_To_Greyscale(im1)
output = Image.fromarray(image)
output.show()

(28, 28)

In [14]:
# Calculates Gradients of Image in X and Y direction
def gradient(image):
    gx = np.zeros(image.shape)
    gx[:, 1:-1] = -image[:, :-2] + image[:, 2:]
    gx[:, 0] = -image[:, 0] + image[:, 1]
    gx[:, -1] = -image[:, -2] + image[:, -1]
    
    gy = np.zeros(image.shape)
    gy[1:-1, :] = image[:-2, :] - image[2:, :]
    gy[0, :] = image[0, :] - image[1, :]
    gy[-1, :] = image[-2, :] - image[-1, :]
    
    return gx,gy

In [42]:
# Calculates Magnitude and Orientation matrices for image
def mag_orient(gx,gy):
    magnitude = np.sqrt(gx**2 + gy**2)
    orientation = np.arctan2(gy, gx) *(180 / np.pi) 
    
    # All the values where angle is below 0 ,I have added +180
    for x in range(28):
        for y in range(28):
            if(orientation[x][y] < 0):
                orientation[x][y] += 180
    
    return magnitude,orientation

In [18]:
def cal_histogram(arr,weights):
    hist,_ = np.histogram(arr,bins=9,range=(0,180),weights=weights)
    return hist
    

In [80]:
# This function assumes the Images are of size 28*28 and thus it returns feature vector of size 1296
def hog_features(mag,orient):
    rows,cols = mag.shape
    cell_array = []
    count = 0
    i = 0
    
    # I have divided the image into 4 pixel*4 pixel cells 
    # Calculates histogram for all cells and stores it in cell_array[]
    while(i < rows-3):
        j = 0
        while(j < cols-3):
            arr = orient[i:i+4,j:j+4]
            weight = mag[i:i+4,j:j+4]
            
            hist = cal_histogram(arr,weight)
            cell_array.append(hist)
            j += 4
        i+=4
        
    cell_array = np.reshape(cell_array,(7, 7, 9))
    
    block = []
    
    i = 0

    # Block size is 2*2 which means 1 block contains total 4 cells in it
    # Histograms of 4 cells in a block are combined then normalized per block and then added to feature vector
    while(i < 6):
        j = 0
        while(j < 6):
            x = cell_array[i:i+2,j:j+2]
            norm1 = np.linalg.norm(x)
            block += (x/(norm1+1e-6)).flatten().tolist() #added 1e-6 because of division by zero
                                                 # error caused by norm being zero sometimes
            j += 1
        i += 1
        
    return block # size is 1296 
            
            

In [61]:
print(len(hog_features(mag,orient)))

36
1296


In [62]:
# Main HOG Function (just requires image as input and return feature vector as output)
def hog(image):
    gx,gy = gradient(image)
    mag,orient = mag_orient(gx,gy)
    features = hog_features(mag,orient)
    return features


1296


60000

In [91]:
dataset = []
for i in range(testX.shape[0]):
    image = np.reshape(testX[i],(28,28))
    features = hog(image)
    dataset += features
    print("Images Done:"+str(i+1))

    

Images Done:1
Images Done:2
Images Done:3
Images Done:4
Images Done:5
Images Done:6
Images Done:7
Images Done:8
Images Done:9
Images Done:10
Images Done:11
Images Done:12
Images Done:13
Images Done:14
Images Done:15
Images Done:16
Images Done:17
Images Done:18
Images Done:19
Images Done:20
Images Done:21
Images Done:22
Images Done:23
Images Done:24
Images Done:25
Images Done:26
Images Done:27
Images Done:28
Images Done:29
Images Done:30
Images Done:31
Images Done:32
Images Done:33
Images Done:34
Images Done:35
Images Done:36
Images Done:37
Images Done:38
Images Done:39
Images Done:40
Images Done:41
Images Done:42
Images Done:43
Images Done:44
Images Done:45
Images Done:46
Images Done:47
Images Done:48
Images Done:49
Images Done:50
Images Done:51
Images Done:52
Images Done:53
Images Done:54
Images Done:55
Images Done:56
Images Done:57
Images Done:58
Images Done:59
Images Done:60
Images Done:61
Images Done:62
Images Done:63
Images Done:64
Images Done:65
Images Done:66
Images Done:67
Imag

Images Done:549
Images Done:550
Images Done:551
Images Done:552
Images Done:553
Images Done:554
Images Done:555
Images Done:556
Images Done:557
Images Done:558
Images Done:559
Images Done:560
Images Done:561
Images Done:562
Images Done:563
Images Done:564
Images Done:565
Images Done:566
Images Done:567
Images Done:568
Images Done:569
Images Done:570
Images Done:571
Images Done:572
Images Done:573
Images Done:574
Images Done:575
Images Done:576
Images Done:577
Images Done:578
Images Done:579
Images Done:580
Images Done:581
Images Done:582
Images Done:583
Images Done:584
Images Done:585
Images Done:586
Images Done:587
Images Done:588
Images Done:589
Images Done:590
Images Done:591
Images Done:592
Images Done:593
Images Done:594
Images Done:595
Images Done:596
Images Done:597
Images Done:598
Images Done:599
Images Done:600
Images Done:601
Images Done:602
Images Done:603
Images Done:604
Images Done:605
Images Done:606
Images Done:607
Images Done:608
Images Done:609
Images Done:610
Images D

Images Done:1068
Images Done:1069
Images Done:1070
Images Done:1071
Images Done:1072
Images Done:1073
Images Done:1074
Images Done:1075
Images Done:1076
Images Done:1077
Images Done:1078
Images Done:1079
Images Done:1080
Images Done:1081
Images Done:1082
Images Done:1083
Images Done:1084
Images Done:1085
Images Done:1086
Images Done:1087
Images Done:1088
Images Done:1089
Images Done:1090
Images Done:1091
Images Done:1092
Images Done:1093
Images Done:1094
Images Done:1095
Images Done:1096
Images Done:1097
Images Done:1098
Images Done:1099
Images Done:1100
Images Done:1101
Images Done:1102
Images Done:1103
Images Done:1104
Images Done:1105
Images Done:1106
Images Done:1107
Images Done:1108
Images Done:1109
Images Done:1110
Images Done:1111
Images Done:1112
Images Done:1113
Images Done:1114
Images Done:1115
Images Done:1116
Images Done:1117
Images Done:1118
Images Done:1119
Images Done:1120
Images Done:1121
Images Done:1122
Images Done:1123
Images Done:1124
Images Done:1125
Images Done:11

Images Done:1566
Images Done:1567
Images Done:1568
Images Done:1569
Images Done:1570
Images Done:1571
Images Done:1572
Images Done:1573
Images Done:1574
Images Done:1575
Images Done:1576
Images Done:1577
Images Done:1578
Images Done:1579
Images Done:1580
Images Done:1581
Images Done:1582
Images Done:1583
Images Done:1584
Images Done:1585
Images Done:1586
Images Done:1587
Images Done:1588
Images Done:1589
Images Done:1590
Images Done:1591
Images Done:1592
Images Done:1593
Images Done:1594
Images Done:1595
Images Done:1596
Images Done:1597
Images Done:1598
Images Done:1599
Images Done:1600
Images Done:1601
Images Done:1602
Images Done:1603
Images Done:1604
Images Done:1605
Images Done:1606
Images Done:1607
Images Done:1608
Images Done:1609
Images Done:1610
Images Done:1611
Images Done:1612
Images Done:1613
Images Done:1614
Images Done:1615
Images Done:1616
Images Done:1617
Images Done:1618
Images Done:1619
Images Done:1620
Images Done:1621
Images Done:1622
Images Done:1623
Images Done:16

Images Done:2055
Images Done:2056
Images Done:2057
Images Done:2058
Images Done:2059
Images Done:2060
Images Done:2061
Images Done:2062
Images Done:2063
Images Done:2064
Images Done:2065
Images Done:2066
Images Done:2067
Images Done:2068
Images Done:2069
Images Done:2070
Images Done:2071
Images Done:2072
Images Done:2073
Images Done:2074
Images Done:2075
Images Done:2076
Images Done:2077
Images Done:2078
Images Done:2079
Images Done:2080
Images Done:2081
Images Done:2082
Images Done:2083
Images Done:2084
Images Done:2085
Images Done:2086
Images Done:2087
Images Done:2088
Images Done:2089
Images Done:2090
Images Done:2091
Images Done:2092
Images Done:2093
Images Done:2094
Images Done:2095
Images Done:2096
Images Done:2097
Images Done:2098
Images Done:2099
Images Done:2100
Images Done:2101
Images Done:2102
Images Done:2103
Images Done:2104
Images Done:2105
Images Done:2106
Images Done:2107
Images Done:2108
Images Done:2109
Images Done:2110
Images Done:2111
Images Done:2112
Images Done:21

...

Images Done:7999
Images Done:8000
Images Done:8001
Images Done:8002
Images Done:8003
Images Done:8004
Images Done:8005
Images Done:8006
Images Done:8007
Images Done:8008
Images Done:8009
Images Done:8010
Images Done:8011
Images Done:8012
Images Done:8013
Images Done:8014
Images Done:8015
Images Done:8016
Images Done:8017
Images Done:8018
Images Done:8019
Images Done:8020
Images Done:8021
Images Done:8022
Images Done:8023
Images Done:8024
Images Done:8025
Images Done:8026
Images Done:8027
Images Done:8028
Images Done:8029
Images Done:8030
Images Done:8031
Images Done:8032
Images Done:8033
Images Done:8034
Images Done:8035
Images Done:8036
Images Done:8037
Images Done:8038
Images Done:8039
Images Done:8040
Images Done:8041
Images Done:8042
Images Done:8043
Images Done:8044
Images Done:8045
Images Done:8046
Images Done:8047
Images Done:8048
Images Done:8049
Images Done:8050
Images Done:8051
Images Done:8052
Images Done:8053
Images Done:8054
Images Done:8055
Images Done:8056
Images Done:80

Images Done:8504
Images Done:8505
Images Done:8506
Images Done:8507
Images Done:8508
Images Done:8509
Images Done:8510
Images Done:8511
Images Done:8512
Images Done:8513
Images Done:8514
Images Done:8515
Images Done:8516
Images Done:8517
Images Done:8518
Images Done:8519
Images Done:8520
Images Done:8521
Images Done:8522
Images Done:8523
Images Done:8524
Images Done:8525
Images Done:8526
Images Done:8527
Images Done:8528
Images Done:8529
Images Done:8530
Images Done:8531
Images Done:8532
Images Done:8533
Images Done:8534
Images Done:8535
Images Done:8536
Images Done:8537
Images Done:8538
Images Done:8539
Images Done:8540
Images Done:8541
Images Done:8542
Images Done:8543
Images Done:8544
Images Done:8545
Images Done:8546
Images Done:8547
Images Done:8548
Images Done:8549
Images Done:8550
Images Done:8551
Images Done:8552
Images Done:8553
Images Done:8554
Images Done:8555
Images Done:8556
Images Done:8557
Images Done:8558
Images Done:8559
Images Done:8560
Images Done:8561
Images Done:85

Images Done:9001
Images Done:9002
Images Done:9003
Images Done:9004
Images Done:9005
Images Done:9006
Images Done:9007
Images Done:9008
Images Done:9009
Images Done:9010
Images Done:9011
Images Done:9012
Images Done:9013
Images Done:9014
Images Done:9015
Images Done:9016
Images Done:9017
Images Done:9018
Images Done:9019
Images Done:9020
Images Done:9021
Images Done:9022
Images Done:9023
Images Done:9024
Images Done:9025
Images Done:9026
Images Done:9027
Images Done:9028
Images Done:9029
Images Done:9030
Images Done:9031
Images Done:9032
Images Done:9033
Images Done:9034
Images Done:9035
Images Done:9036
Images Done:9037
Images Done:9038
Images Done:9039
Images Done:9040
Images Done:9041
Images Done:9042
Images Done:9043
Images Done:9044
Images Done:9045
Images Done:9046
Images Done:9047
Images Done:9048
Images Done:9049
Images Done:9050
Images Done:9051
Images Done:9052
Images Done:9053
Images Done:9054
Images Done:9055
Images Done:9056
Images Done:9057
Images Done:9058
Images Done:90

Images Done:9492
Images Done:9493
Images Done:9494
Images Done:9495
Images Done:9496
Images Done:9497
Images Done:9498
Images Done:9499
Images Done:9500
Images Done:9501
Images Done:9502
Images Done:9503
Images Done:9504
Images Done:9505
Images Done:9506
Images Done:9507
Images Done:9508
Images Done:9509
Images Done:9510
Images Done:9511
Images Done:9512
Images Done:9513
Images Done:9514
Images Done:9515
Images Done:9516
Images Done:9517
Images Done:9518
Images Done:9519
Images Done:9520
Images Done:9521
Images Done:9522
Images Done:9523
Images Done:9524
Images Done:9525
Images Done:9526
Images Done:9527
Images Done:9528
Images Done:9529
Images Done:9530
Images Done:9531
Images Done:9532
Images Done:9533
Images Done:9534
Images Done:9535
Images Done:9536
Images Done:9537
Images Done:9538
Images Done:9539
Images Done:9540
Images Done:9541
Images Done:9542
Images Done:9543
Images Done:9544
Images Done:9545
Images Done:9546
Images Done:9547
Images Done:9548
Images Done:9549
Images Done:95

Images Done:9988
Images Done:9989
Images Done:9990
Images Done:9991
Images Done:9992
Images Done:9993
Images Done:9994
Images Done:9995
Images Done:9996
Images Done:9997
Images Done:9998
Images Done:9999
Images Done:10000


In [92]:
dataset = np.reshape(dataset,(10000,1296))

In [93]:
np.append(dataset,test.loc[:,test.columns == 'label'].to_numpy(),axis=1)

array([[6.19628352e-03, 3.07895804e-03, 1.94730404e-03, ...,
        0.00000000e+00, 1.34465924e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.83203403e-02, 2.44203244e-01, 2.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.32058156e-01, 4.88588079e-01, 8.00000000e+00],
       [2.17189737e-03, 0.00000000e+00, 0.00000000e+00, ...,
        9.18537784e-02, 2.91844770e-03, 8.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.59601032e-01, 1.00000000e+00]])

In [94]:
np.savetxt('hog_test_features.csv', dataset, delimiter=',')